In [1]:
import pandas as pd
pd.options.display.max_colwidth = 200
import plotly.express as px
import re
import datetime
import numpy as np
import json
from Project.Database import Db
from Project._04TPMAlgorithm.transform_for_TPM_algorithm import light_location_dict
from Project._05InferKnowledgeOfRules.infer_rules_functions import json_to_dataframe, redundancy_filter_tool

In [2]:
level1_df = json_to_dataframe(year=1,level=1)

In [3]:
level2_df = json_to_dataframe(year=1,level=2)

/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/_05InferKnowledgeOfRules/infer_rules_functions.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.dropna(inplace=True, axis=0)


In [4]:
level3_df = json_to_dataframe(year=1,level=3)

/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/_05InferKnowledgeOfRules/infer_rules_functions.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.dropna(inplace=True, axis=0)


In [5]:
meta = Db.load_data(meta=True, consumption=False, hourly=False)

In [6]:
def filter_rule_indexes(dataframe):
    level_3_check = len(re.findall('\*', dataframe.loc[0, 'pattern'])) > 0
    rule_type = ['app_app_app', 'psn_app_app', 'psn_psn_app']

    #filter follows rules in dataframe for level3: max 1 and level2: 0
    if level_3_check:
        return_dict = {
            'multi_floor': {'app_app_app': [],'psn_app_app': [],'psn_psn_app': []},
            'single_floor': {'app_app_app': [],'psn_app_app': [],'psn_psn_app': []}
        }
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) <= 1]
    else:
        return_dict = {
            'multi_floor': {'app_app': [],'psn_app': []},
            'single_floor': {'app_app': [],'psn_app': []}
        }
        dataframe = dataframe.loc[dataframe['pattern'].str.findall('-').map(len) == 0]

    for index, row in dataframe.iterrows():
        tmp_floor_set = set()
        appliance_check_list = list()
        person_check_list = list()
        for col in set(re.findall('[\w_]+', row['pattern'])):
            person_check_list.append('SensHeat' in col)
            appliance_check_list.append('SensHeat' not in col)
            try:
                tmp_floor_set.add(meta.loc[col, 'Measurement_Floor'])
            except:
                tmp_floor_set.add(light_location_dict(meta)[col][0])

        if sum(appliance_check_list) >= 1:
            if {'1stFloor', '2ndFloor'} == tmp_floor_set:
                if level_3_check:
                    return_dict['multi_floor'][rule_type[sum(person_check_list)]].append(index)
                else:
                    return_dict['multi_floor'][rule_type[sum(person_check_list)][:-4]].append(index)
            else:
                if level_3_check:
                    return_dict['single_floor'][rule_type[sum(person_check_list)]].append(index)
                else:
                    return_dict['single_floor'][rule_type[sum(person_check_list)][:-4]].append(index)

    return return_dict

In [7]:
level2_index_dict = filter_rule_indexes(level2_df)
level2_index_dict

{'multi_floor': {'app_app': [1,
   2,
   7,
   9,
   10,
   62,
   67,
   69,
   71,
   89,
   90,
   96,
   100,
   104,
   108,
   109,
   110,
   111,
   120,
   121,
   159,
   167,
   168,
   172,
   182,
   183,
   184,
   187,
   263,
   268,
   275,
   282,
   285,
   286,
   287,
   290,
   297,
   298,
   299,
   301,
   304,
   305,
   312,
   313,
   318,
   319,
   342,
   353],
  'psn_app': [0,
   11,
   15,
   19,
   21,
   22,
   28,
   29,
   72,
   93,
   107,
   118,
   119,
   123,
   124,
   127,
   130,
   132,
   133,
   134,
   138,
   139,
   144,
   145,
   156,
   160,
   161,
   173,
   174,
   191,
   192,
   194,
   197,
   199,
   201,
   225,
   226,
   228,
   230,
   232,
   236,
   238,
   239,
   240,
   243,
   244,
   245,
   250,
   251,
   253,
   255,
   257,
   259,
   260,
   261,
   273,
   276,
   277,
   291,
   292,
   303,
   310,
   311,
   314,
   315,
   323,
   324,
   326,
   328,
   330,
   334,
   336,
   337]},
 'single_floor': {'

In [8]:
level3_index_dict = filter_rule_indexes(level3_df)

In [9]:
# def redundancy_filter_tool(dataframe=level3_df, regex_str='BR3[\w]*>[\w]*ChildB', rule_type='psn_app_app', multi_floor=True):
#     multi_floor_selector = 'multi_floor' if multi_floor else 'single_floor'
#     dataframe = dataframe.loc[filter_rule_indexes(dataframe)[multi_floor_selector][rule_type]]
#     return dataframe.loc[dataframe['pattern'].str.findall(regex_str).map(len) > 0]

In [10]:
level2_df.head()

,pattern,supp,conf,time,rule,multi_floor
0,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'21': [[['2013-07-22 05:59:52', '2013-07-22 06:14:52'], ['2013-07-22 05:59:52', '2013-07-22 06:29:52']]], '1': [[['2013-07-02 05:58:29', '2013-07-02 06:13:29'], ['2013-07-02 05:58:29', '2013-07-0...",psn_app,True
1,Load_StatusMBALights>Load_StatusPlugLoadBlender,0.701370,0.707182,"{'21': [[['2013-07-22 05:59:52', '2013-07-22 06:14:52'], ['2013-07-22 06:04:52', '2013-07-22 06:11:52']]], '1': [[['2013-07-02 05:58:29', '2013-07-02 06:13:29'], ['2013-07-02 06:03:29', '2013-07-0...",app_app,True
2,Load_StatusMBALights>Load_StatusPlugLoadCoffeeMaker,0.701370,0.707182,"{'21': [[['2013-07-22 05:59:52', '2013-07-22 06:14:52'], ['2013-07-22 06:09:52', '2013-07-22 06:27:52']]], '1': [[['2013-07-02 05:58:29', '2013-07-02 06:13:29'], ['2013-07-02 06:08:29', '2013-07-0...",app_app,True
3,Load_StatusMBALights>Load_StatusBA2Lights,0.701370,0.707182,"{'21': [[['2013-07-22 06:29:52', '2013-07-22 06:44:52'], ['2013-07-22 06:29:52', '2013-07-22 06:49:52']]], '1': [[['2013-07-02 06:28:29', '2013-07-02 06:43:29'], ['2013-07-02 06:28:29', '2013-07-0...",app_app,False
4,Load_StatusMBALights>Load_StatusSensHeatPrntBUP,0.841096,0.841096,"{'21': [[['2013-07-22 06:29:52', '2013-07-22 06:44:52'], ['2013-07-22 06:29:52', '2013-07-22 06:59:52']]], '236': [[['2014-02-22 23:28:08', '2014-02-22 23:43:08'], ['2014-02-22 23:28:08', '2014-02...",psn_app,False


In [11]:
ownership1_true = redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*>[\w]*SensHeat', rule_type='psn_app', multi_floor=True)
ownership1_false = redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*>[\w]*SensHeat', rule_type='psn_app', multi_floor=False)
ownership1_false

,pattern,supp,conf,time,rule,multi_floor
4,Load_StatusMBALights>Load_StatusSensHeatPrntBUP,0.841096,0.841096,"{'21': [[['2013-07-22 06:29:52', '2013-07-22 06:44:52'], ['2013-07-22 06:29:52', '2013-07-22 06:59:52']]], '236': [[['2014-02-22 23:28:08', '2014-02-22 23:43:08'], ['2014-02-22 23:28:08', '2014-02...",psn_app,False
8,Load_StatusMBALights>Load_StatusSensHeatPrntAUP,0.556164,0.556164,"{'188': [[['2014-01-05 13:59:07', '2014-01-05 14:14:07'], ['2014-01-05 13:59:07', '2014-01-05 14:29:07']]], '236': [[['2014-02-22 19:58:08', '2014-02-22 20:13:08'], ['2014-02-22 19:58:08', '2014-0...",psn_app,False
34,Load_StatusKitchenLights>Load_StatusSensHeatPrntBDOWN,0.986301,0.994475,"{'188': [[['2014-01-05 08:14:07', '2014-01-05 09:59:07'], ['2014-01-05 08:14:07', '2014-01-05 09:59:07']]], '21': [[['2013-07-22 05:59:52', '2013-07-22 08:24:52'], ['2013-07-22 05:59:52', '2013-07...",psn_app,False
44,Load_StatusKitchenLights>Load_StatusSensHeatPrntADOWN,0.983562,0.991713,"{'188': [[['2014-01-05 08:14:07', '2014-01-05 09:59:07'], ['2014-01-05 08:14:07', '2014-01-05 09:59:07']]], '21': [[['2013-07-22 05:59:52', '2013-07-22 08:24:52'], ['2013-07-22 06:29:52', '2013-07...",psn_app,False
46,Load_StatusKitchenLights>Load_StatusSensHeatPrntBUP,0.701370,0.701370,"{'21': [[['2013-07-22 05:59:52', '2013-07-22 08:24:52'], ['2013-07-22 06:29:52', '2013-07-22 06:59:52']]], '1': [[['2013-07-02 05:58:29', '2013-07-02 08:23:29'], ['2013-07-02 06:28:29', '2013-07-0...",psn_app,False
52,Load_StatusKitchenLights>Load_StatusSensHeatChildADOWN,0.991781,1.000000,"{'188': [[['2014-01-05 19:29:06', '2014-01-05 20:14:06'], ['2014-01-05 19:29:06', '2014-01-05 20:14:06']]], '21': [[['2013-07-22 05:59:52', '2013-07-22 08:24:52'], ['2013-07-22 06:59:52', '2013-07...",psn_app,False
53,Load_StatusKitchenLights>Load_StatusSensHeatChildBDOWN,0.991781,1.000000,"{'188': [[['2014-01-05 08:14:07', '2014-01-05 09:59:07'], ['2014-01-05 09:29:07', '2014-01-05 09:59:07']], [['2014-01-05 19:29:06', '2014-01-05 20:14:06'], ['2014-01-05 19:29:06', '2014-01-05 20:1...",psn_app,False
56,Load_StatusKitchenLights>Load_StatusSensHeatPrntAUP,0.983562,0.983562,"{'188': [[['2014-01-05 17:59:06', '2014-01-05 18:29:06'], ['2014-01-05 17:59:06', '2014-01-05 18:29:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 18:29:51'], ['2013-07-22 17:59:51', '2013-07...",psn_app,False
73,Load_StatusPlugLoadMBRTV>Load_StatusSensHeatPrntBUP,0.843836,0.843836,"{'188': [[['2014-01-05 21:59:06', '2014-01-05 23:29:06'], ['2014-01-05 21:59:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 06:14:52', '2013-07-22 06:59:52'], ['2013-07-22 06:29:52', '2013-07...",psn_app,False
97,Load_StatusBA2Lights>Load_StatusSensHeatPrntBUP,0.701370,0.701370,"{'21': [[['2013-07-22 06:29:52', '2013-07-22 06:49:52'], ['2013-07-22 06:29:52', '2013-07-22 06:59:52']]], '1': [[['2013-07-02 06:28:29', '2013-07-02 06:48:29'], ['2013-07-02 06:28:29', '2013-07-0...",psn_app,False


In [12]:
ownership2_false = redundancy_filter_tool(dataframe=level2_df, regex_str='SensHeat[\w]*>[\w]*', rule_type='psn_app', multi_floor=False)
ownership2_false

,pattern,supp,conf,time,rule,multi_floor
12,Load_StatusSensHeatPrntBDOWN>Load_StatusKitchenLights,0.846575,0.853591,"{'188': [[['2014-01-05 16:59:07', '2014-01-05 21:59:06'], ['2014-01-05 17:59:06', '2014-01-05 18:29:06']], [['2014-01-05 16:59:07', '2014-01-05 21:59:06'], ['2014-01-05 18:59:06', '2014-01-05 19:1...",psn_app,False
13,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadBlender,0.980822,0.988950,"{'188': [[['2014-01-05 08:14:07', '2014-01-05 09:59:07'], ['2014-01-05 09:14:07', '2014-01-05 09:20:07']]], '21': [[['2013-07-22 05:59:52', '2013-07-22 06:29:52'], ['2013-07-22 06:04:52', '2013-07...",psn_app,False
14,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadCoffeeMaker,0.980822,0.988950,"{'188': [[['2014-01-05 08:14:07', '2014-01-05 09:59:07'], ['2014-01-05 09:09:07', '2014-01-05 09:27:07']]], '21': [[['2013-07-22 05:59:52', '2013-07-22 06:29:52'], ['2013-07-22 06:09:52', '2013-07...",psn_app,False
16,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadToasterOven,0.980822,0.988950,"{'188': [[['2014-01-05 08:14:07', '2014-01-05 09:59:07'], ['2014-01-05 09:19:07', '2014-01-05 09:29:07']]], '21': [[['2013-07-22 05:59:52', '2013-07-22 06:29:52'], ['2013-07-22 06:14:52', '2013-07...",psn_app,False
17,Load_StatusSensHeatPrntBDOWN>Load_StatusMasterBedroomLights,0.989041,0.997238,"{'188': [[['2014-01-05 16:59:07', '2014-01-05 21:59:06'], ['2014-01-05 18:06:06', '2014-01-05 18:29:06']]], '21': [[['2013-07-22 05:59:52', '2013-07-22 06:29:52'], ['2013-07-22 06:14:52', '2013-07...",psn_app,False
...,...,...,...,...,...,...
332,Load_StatusSensHeatChildBUP>Load_StatusPlugLoadHairDryerCurlIron,0.687671,0.687671,"{'21': [[['2013-07-22 00:04:50', '2013-07-22 06:59:52'], ['2013-07-22 06:39:52', '2013-07-22 06:44:52']]], '182': [[['2013-12-30 00:05:00', '2013-12-30 07:00:05'], ['2013-12-30 06:40:05', '2013-12...",psn_app,False
333,Load_StatusSensHeatChildBUP>Load_StatusBR3Lights,0.835616,0.835616,"{'21': [[['2013-07-22 00:04:50', '2013-07-22 06:59:52'], ['2013-07-22 06:49:52', '2013-07-22 06:59:52']], [['2013-07-22 20:14:51', '2013-07-22 23:58:51'], ['2013-07-22 20:14:51', '2013-07-22 20:29...",psn_app,False
335,Load_StatusSensHeatChildBUP>Load_StatusLivingRoomLights,0.704110,0.704110,"{'188': [[['2014-01-05 18:29:06', '2014-01-05 19:29:06'], ['2014-01-05 18:59:06', '2014-01-05 19:14:06']]], '1': [[['2013-07-02 18:28:28', '2013-07-02 19:28:28'], ['2013-07-02 18:58:28', '2013-07-...",psn_app,False
338,Load_StatusSensHeatChildBUP>Load_StatusPlugLoadFan,0.978082,0.978082,"{'188': [[['2014-01-05 20:14:06', '2014-01-05 23:58:06'], ['2014-01-05 21:59:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 20:14:51', '2013-07-22 23:58:51'], ['2013-07-22 21:59:51', '2013-07...",psn_app,False


In [13]:
ownership3 = redundancy_filter_tool(dataframe=level2_df, regex_str='SensHeat[\w]*\|[\w]*', rule_type='psn_app', multi_floor=False)
ownership3

,pattern,supp,conf,time,rule,multi_floor
200,Load_StatusSensHeatChildBDOWN|Load_StatusPlugLoadLRTV,0.843836,0.850829,"{'188': [[['2014-01-05 12:59:07', '2014-01-05 18:29:06'], ['2014-01-05 18:09:06', '2014-01-05 22:39:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 18:29:51'], ['2013-07-22 18:09:51', '2013-07...",psn_app,False
223,Load_StatusSensHeatChildAUP|Load_StatusKitchenLights,0.967123,0.967123,"{'188': [[['2014-01-05 00:04:06', '2014-01-05 08:59:07'], ['2014-01-05 08:14:07', '2014-01-05 09:59:07']]], '21': [[['2013-07-22 00:04:50', '2013-07-22 06:59:52'], ['2013-07-22 05:59:52', '2013-07...",psn_app,False
241,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadBR2Laptop,0.558904,0.558904,"{'188': [[['2014-01-05 17:59:06', '2014-01-05 19:29:06'], ['2014-01-05 18:29:06', '2014-01-05 21:59:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 19:29:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,False
242,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadBR3Laptop,0.558904,0.558904,"{'188': [[['2014-01-05 17:59:06', '2014-01-05 19:29:06'], ['2014-01-05 18:29:06', '2014-01-05 20:14:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 19:29:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,False
249,Load_StatusSensHeatPrntAUP|Load_StatusKitchenLights,0.687671,0.687671,"{'21': [[['2013-07-22 00:04:50', '2013-07-22 06:29:52'], ['2013-07-22 05:59:52', '2013-07-22 08:24:52']]], '182': [[['2013-12-30 00:05:00', '2013-12-30 06:30:05'], ['2013-12-30 06:00:05', '2013-12...",psn_app,False
258,Load_StatusSensHeatPrntAUP|Load_StatusBR2Lights,0.558904,0.558904,"{'188': [[['2014-01-05 17:59:06', '2014-01-05 18:29:06'], ['2014-01-05 18:19:06', '2014-01-06 00:00:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 18:29:51'], ['2013-07-22 18:19:51', '2013-07...",psn_app,False
321,Load_StatusSensHeatChildBUP|Load_StatusKitchenLights,0.967123,0.967123,"{'188': [[['2014-01-05 00:04:06', '2014-01-05 08:59:07'], ['2014-01-05 08:14:07', '2014-01-05 09:59:07']]], '21': [[['2013-07-22 00:04:50', '2013-07-22 06:59:52'], ['2013-07-22 05:59:52', '2013-07...",psn_app,False


In [14]:
ownership4 = redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*\|[\w]*', rule_type='psn_app', multi_floor=True)
ownership4

,pattern,supp,conf,time,rule,multi_floor
72,Load_StatusPlugLoadMBRTV|Load_StatusSensHeatPrntADOWN,0.701370,0.707182,"{'21': [[['2013-07-22 06:14:52', '2013-07-22 06:59:52'], ['2013-07-22 06:29:52', '2013-07-22 08:29:52']]], '1': [[['2013-07-02 06:13:29', '2013-07-02 06:58:29'], ['2013-07-02 06:28:29', '2013-07-0...",psn_app,True
118,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'188': [[['2014-01-05 18:09:06', '2014-01-05 22:39:06'], ['2014-01-05 20:14:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 18:09:51', '2013-07-22 20:29:51'], ['2013-07-22 20:14:51', '2013-07...",psn_app,True
123,Load_StatusPlugLoadLRTV|Load_StatusSensHeatChildBUP,0.838356,0.838356,"{'188': [[['2014-01-05 18:09:06', '2014-01-05 22:39:06'], ['2014-01-05 20:14:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 18:09:51', '2013-07-22 20:29:51'], ['2013-07-22 20:14:51', '2013-07...",psn_app,True
144,Load_StatusSensHeatPrntADOWN|Load_StatusPlugLoadFan,0.698630,0.704420,"{'188': [[['2014-01-05 18:29:06', '2014-01-05 22:29:06'], ['2014-01-05 21:59:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 18:29:51', '2013-07-22 22:29:51'], ['2013-07-22 21:59:51', '2013-07...",psn_app,True
191,Load_StatusDRLights|Load_StatusSensHeatChildAUP,0.701370,0.701370,"{'188': [[['2014-01-05 19:14:06', '2014-01-05 20:29:06'], ['2014-01-05 20:14:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 19:14:51', '2013-07-22 20:29:51'], ['2013-07-22 20:14:51', '2013-07...",psn_app,True
192,Load_StatusDRLights|Load_StatusSensHeatChildBUP,0.701370,0.701370,"{'188': [[['2014-01-05 19:14:06', '2014-01-05 20:29:06'], ['2014-01-05 20:14:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 19:14:51', '2013-07-22 20:29:51'], ['2013-07-22 20:14:51', '2013-07...",psn_app,True
201,Load_StatusSensHeatChildBDOWN|Load_StatusBR2Lights,0.558904,0.563536,"{'188': [[['2014-01-05 12:59:07', '2014-01-05 18:29:06'], ['2014-01-05 18:19:06', '2014-01-06 00:00:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 18:29:51'], ['2013-07-22 18:19:51', '2013-07...",psn_app,True
232,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadLRTV,0.975342,0.975342,"{'188': [[['2014-01-05 00:04:06', '2014-01-05 08:59:07'], ['2014-01-05 08:29:07', '2014-01-05 10:29:07']], [['2014-01-05 17:59:06', '2014-01-05 19:29:06'], ['2014-01-05 18:09:06', '2014-01-05 22:3...",psn_app,True
236,Load_StatusSensHeatChildAUP|Load_StatusDRLights,0.843836,0.843836,"{'188': [[['2014-01-05 17:59:06', '2014-01-05 19:29:06'], ['2014-01-05 19:14:06', '2014-01-05 20:29:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 19:29:51'], ['2013-07-22 19:14:51', '2013-07...",psn_app,True
243,Load_StatusSensHeatChildAUP|Load_StatusPlugLoadDesktopPCMonitor,0.698630,0.698630,"{'188': [[['2014-01-05 00:04:06', '2014-01-05 08:59:07'], ['2014-01-05 08:29:07', '2014-01-05 18:28:06']]], '21': [[['2013-07-22 17:59:51', '2013-07-22 19:29:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,True


In [15]:
def potential_redundant_time_intervals(dataframe):
    rule_potential_redundancy = {}
    for index, row in psn_app_df.iterrows():
        rule_potential_redundancy[row['pattern']] = []
        for time_intervals in row['time'].values():
            for time_interval in time_intervals:
                rule_potential_redundancy[row['pattern']].append(time_interval[1])
    return rule_potential_redundancy

In [16]:
psn_app_df = redundancy_filter_tool(dataframe=level2_df, regex_str='[\w]*>[\w]*SensHeat', rule_type='psn_app', multi_floor=True)
psn_app_df.loc[~psn_app_df['pattern'].str.contains('Toaster')]

,pattern,supp,conf,time,rule,multi_floor
0,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN,0.704110,0.709945,"{'21': [[['2013-07-22 05:59:52', '2013-07-22 06:14:52'], ['2013-07-22 05:59:52', '2013-07-22 06:29:52']]], '1': [[['2013-07-02 05:58:29', '2013-07-02 06:13:29'], ['2013-07-02 05:58:29', '2013-07-0...",psn_app,True
107,Load_StatusPlugLoadLRTV>Load_StatusSensHeatPrntBUP,0.838356,0.838356,"{'188': [[['2014-01-05 08:29:07', '2014-01-05 10:29:07'], ['2014-01-05 09:59:07', '2014-01-05 10:59:07']]], '21': [[['2013-07-22 06:29:52', '2013-07-22 08:24:52'], ['2013-07-22 06:29:52', '2013-07...",psn_app,True
119,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildAUP,0.564384,0.564384,"{'236': [[['2014-02-22 19:58:08', '2014-02-22 23:28:08'], ['2014-02-22 22:58:08', '2014-02-22 23:57:08']]], '1': [[['2013-07-02 18:08:28', '2013-07-02 20:28:28'], ['2013-07-02 18:28:28', '2013-07-...",psn_app,True
124,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildBUP,0.983562,0.983562,"{'188': [[['2014-01-05 18:09:06', '2014-01-05 22:39:06'], ['2014-01-05 18:29:06', '2014-01-05 19:29:06']]], '21': [[['2013-07-22 18:09:51', '2013-07-22 20:29:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,True
156,Load_StatusBR2Lights>Load_StatusSensHeatPrntADOWN,0.567123,0.571823,"{'188': [[['2014-01-05 18:19:06', '2014-01-06 00:00:06'], ['2014-01-05 18:29:06', '2014-01-05 22:29:06']]], '1': [[['2013-07-02 18:28:28', '2013-07-02 21:58:28'], ['2013-07-02 18:28:28', '2013-07-...",psn_app,True
160,Load_StatusBR2Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'188': [[['2014-01-05 18:19:06', '2014-01-06 00:00:06'], ['2014-01-05 19:29:06', '2014-01-05 20:14:06']]], '1': [[['2013-07-02 18:28:28', '2013-07-02 21:58:28'], ['2013-07-02 19:28:28', '2013-07-...",psn_app,True
161,Load_StatusBR2Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'188': [[['2014-01-05 18:19:06', '2014-01-06 00:00:06'], ['2014-01-05 19:29:06', '2014-01-05 20:14:06']]], '1': [[['2013-07-02 18:28:28', '2013-07-02 21:58:28'], ['2013-07-02 19:28:28', '2013-07-...",psn_app,True
173,Load_StatusBR3Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'188': [[['2014-01-05 18:29:06', '2014-01-05 20:29:06'], ['2014-01-05 19:29:06', '2014-01-05 20:14:06']]], '1': [[['2013-07-02 18:28:28', '2013-07-02 20:28:28'], ['2013-07-02 19:28:28', '2013-07-...",psn_app,True
174,Load_StatusBR3Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'188': [[['2014-01-05 18:29:06', '2014-01-05 20:29:06'], ['2014-01-05 19:29:06', '2014-01-05 20:14:06']]], '1': [[['2013-07-02 18:28:28', '2013-07-02 20:28:28'], ['2013-07-02 19:28:28', '2013-07-...",psn_app,True
273,Load_StatusPlugLoadBR2Laptop>Load_StatusSensHeatPrntADOWN,0.706849,0.712707,"{'188': [[['2014-01-05 18:29:06', '2014-01-05 21:59:06'], ['2014-01-05 18:29:06', '2014-01-05 22:29:06']]], '21': [[['2013-07-22 18:29:51', '2013-07-22 21:59:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,True


In [17]:
potential_redundant_time_intervals(psn_app_df)

{'Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN': [['2013-07-22 05:59:52',
   '2013-07-22 06:29:52'],
  ['2013-07-02 05:58:29', '2013-07-02 06:28:29'],
  ['2013-12-30 06:00:05', '2013-12-30 06:30:05'],
  ['2013-12-04 05:59:06', '2013-12-04 06:29:06'],
  ['2014-06-24 05:58:44', '2014-06-24 06:28:44'],
  ['2013-07-08 05:58:27', '2013-07-08 06:28:27'],
  ['2013-07-12 06:00:01', '2013-07-12 06:30:01'],
  ['2013-11-07 05:59:59', '2013-11-07 06:29:59'],
  ['2013-08-19 05:59:31', '2013-08-19 06:29:31'],
  ['2014-06-02 05:58:24', '2014-06-02 06:28:24'],
  ['2014-02-28 05:58:26', '2014-02-28 06:28:26'],
  ['2014-04-23 05:59:03', '2014-04-23 06:29:03'],
  ['2014-03-28 05:58:16', '2014-03-28 06:28:16'],
  ['2013-07-16 05:59:55', '2013-07-16 06:29:55'],
  ['2014-03-27 05:58:12', '2014-03-27 06:28:12'],
  ['2013-12-09 05:59:24', '2013-12-09 06:29:24'],
  ['2013-08-26 05:59:29', '2013-08-26 06:29:29'],
  ['2014-02-05 05:58:33', '2014-02-05 06:28:32'],
  ['2013-10-28 05:59:52', '2013-10-28 06:29:

In [18]:
psn_app_app_df = redundancy_filter_tool(dataframe=level3_df, regex_str='[\w]*>[\w]*SensHeat', rule_type='psn_app_app', multi_floor=True)

In [25]:
psn_app_app_df

,pattern,supp,conf,time,rule,multi_floor
0,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadBlender*Load_StatusMBALights>Load_StatusPlugLoadBlender,0.701370,0.707182,"{'268': [[['2014-03-26 05:58:08', '2014-03-26 06:13:08'], ['2014-03-26 05:58:08', '2014-03-26 06:28:08'], ['2014-03-26 06:03:08', '2014-03-26 06:10:08']]], '259': [[['2014-03-17 05:58:34', '2014-0...",psn_app_app,True
1,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadCoffeeMaker*Load_StatusMBALights>Load_StatusPlugLoadCoffeeMaker,0.701370,0.707182,"{'268': [[['2014-03-26 05:58:08', '2014-03-26 06:13:08'], ['2014-03-26 05:58:08', '2014-03-26 06:28:08'], ['2014-03-26 06:08:08', '2014-03-26 06:26:08']]], '259': [[['2014-03-17 05:58:34', '2014-0...",psn_app_app,True
2,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadMBRTV*Load_StatusMBALights->Load_StatusPlugLoadMBRTV,0.698630,0.704420,"{'268': [[['2014-03-26 05:58:08', '2014-03-26 06:13:08'], ['2014-03-26 05:58:08', '2014-03-26 06:28:08'], ['2014-03-26 06:13:08', '2014-03-26 06:58:08']]], '259': [[['2014-03-17 05:58:34', '2014-0...",psn_app_app,True
3,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadToasterOven*Load_StatusMBALights->Load_StatusPlugLoadToasterOven,0.701370,0.707182,"{'268': [[['2014-03-26 05:58:08', '2014-03-26 06:13:08'], ['2014-03-26 05:58:08', '2014-03-26 06:28:08'], ['2014-03-26 06:13:08', '2014-03-26 06:23:08']]], '259': [[['2014-03-17 05:58:34', '2014-0...",psn_app_app,True
5,Load_StatusMBALights>Load_StatusSensHeatPrntBDOWN*Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadToaster*Load_StatusMBALights->Load_StatusPlugLoadToaster,0.701370,0.707182,"{'268': [[['2014-03-26 05:58:08', '2014-03-26 06:13:08'], ['2014-03-26 05:58:08', '2014-03-26 06:28:08'], ['2014-03-26 06:18:08', '2014-03-26 06:29:08']]], '259': [[['2014-03-17 05:58:34', '2014-0...",psn_app_app,True
...,...,...,...,...,...,...
2543,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildBUP*Load_StatusSensHeatChildBUP|Load_StatusDRLights*Load_StatusPlugLoadDesktopPCMonitor>Load_StatusDRLights,0.701370,0.707182,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 22:13:07'], ['2014-03-26 18:28:07', '2014-03-26 19:28:07'], ['2014-03-26 19:13:07', '2014-03-26 20:28:07']]], '259': [[['2014-03-17 18:28:33', '2014-0...",psn_app_app,True
2548,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildBUP*Load_StatusSensHeatChildBUP>Load_StatusPlugLoadCanOpener*Load_StatusPlugLoadDesktopPCMonitor>Load_StatusPlugLoadCanOpener,0.726027,0.734072,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 22:13:07'], ['2014-03-26 18:28:07', '2014-03-26 19:28:07'], ['2014-03-26 18:38:07', '2014-03-26 18:45:07']]], '259': [[['2014-03-17 18:28:33', '2014-0...",psn_app_app,True
2549,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildBUP*Load_StatusSensHeatChildBUP>Load_StatusPlugLoadHandMixer*Load_StatusPlugLoadDesktopPCMonitor>Load_StatusPlugLoadHandMixer,0.726027,0.734072,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 22:13:07'], ['2014-03-26 18:28:07', '2014-03-26 19:28:07'], ['2014-03-26 18:38:07', '2014-03-26 18:40:07']]], '259': [[['2014-03-17 18:28:33', '2014-0...",psn_app_app,True
2561,Load_StatusPlugLoadDesktopPCMonitor|Load_StatusPlugLoadFan*Load_StatusPlugLoadFan>Load_StatusSensHeatPrntBUP*Load_StatusPlugLoadDesktopPCMonitor|Load_StatusSensHeatPrntBUP,0.547945,0.547945,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 22:13:07'], ['2014-03-26 21:58:07', '2014-03-26 23:57:07'], ['2014-03-26 21:58:07', '2014-03-26 23:57:07']]], '259': [[['2014-03-17 18:28:33', '2014-0...",psn_app_app,True


In [20]:
psn_psn_app_df = redundancy_filter_tool(dataframe=level3_df, regex_str='MBR[\w]*>[\w]*Prnt', rule_type='psn_psn_app', multi_floor=True)

In [21]:
psn_psn_app_df

,pattern,supp,conf,time,rule,multi_floor
102,Load_StatusSensHeatPrntBDOWN>Load_StatusPlugLoadMBRTV*Load_StatusPlugLoadMBRTV>Load_StatusSensHeatPrntBUP*Load_StatusSensHeatPrntBDOWN->Load_StatusSensHeatPrntBUP,0.69863,0.69863,"{'268': [[['2014-03-26 05:58:08', '2014-03-26 06:28:08'], ['2014-03-26 06:13:08', '2014-03-26 06:58:08'], ['2014-03-26 06:28:08', '2014-03-26 06:58:08']]], '259': [[['2014-03-17 05:58:34', '2014-0...",psn_psn_app,True
766,Load_StatusPlugLoadMBRTV|Load_StatusSensHeatPrntADOWN*Load_StatusSensHeatPrntADOWN>Load_StatusSensHeatPrntBUP*Load_StatusPlugLoadMBRTV>Load_StatusSensHeatPrntBUP,0.70137,0.70137,"{'268': [[['2014-03-26 06:13:08', '2014-03-26 06:58:08'], ['2014-03-26 06:28:08', '2014-03-26 08:28:08'], ['2014-03-26 06:28:08', '2014-03-26 06:58:08']]], '259': [[['2014-03-17 06:13:34', '2014-0...",psn_psn_app,True


In [22]:
level3_df.loc[level3_df['pattern'].str.contains('VideoGame')]

,pattern,supp,conf,time,rule,multi_floor


# Creating a list of redundant rules
## First, look at ownership in level2 for same floor

In [23]:
appliance_ownership = pd.DataFrame()
appliance_ownership = pd.concat([appliance_ownership, ownership1_true.loc[[119,124,156,273,311,315]]], ignore_index=True, axis=0)
appliance_ownership = pd.concat([appliance_ownership, ownership1_false.loc[[73,101,106,113,114,166,178,181,280,284,295,296,302,307,308,345]]], ignore_index=True, axis=0)
appliance_ownership = pd.concat([appliance_ownership, ownership2_false.loc[[13,14,16,18,20,26,27,30,31,32,131,140,141,142,143,150,152,203,204,252]]], ignore_index=True, axis=0)
appliance_ownership = pd.concat([appliance_ownership, ownership3.loc[[241,242]]], ignore_index=True, axis=0)

appliance_ownership

,pattern,supp,conf,time,rule,multi_floor
0,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildAUP,0.564384,0.564384,"{'236': [[['2014-02-22 19:58:08', '2014-02-22 23:28:08'], ['2014-02-22 22:58:08', '2014-02-22 23:57:08']]], '1': [[['2013-07-02 18:08:28', '2013-07-02 20:28:28'], ['2013-07-02 18:28:28', '2013-07-...",psn_app,True
1,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildBUP,0.983562,0.983562,"{'188': [[['2014-01-05 18:09:06', '2014-01-05 22:39:06'], ['2014-01-05 18:29:06', '2014-01-05 19:29:06']]], '21': [[['2013-07-22 18:09:51', '2013-07-22 20:29:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,True
2,Load_StatusBR2Lights>Load_StatusSensHeatPrntADOWN,0.567123,0.571823,"{'188': [[['2014-01-05 18:19:06', '2014-01-06 00:00:06'], ['2014-01-05 18:29:06', '2014-01-05 22:29:06']]], '1': [[['2013-07-02 18:28:28', '2013-07-02 21:58:28'], ['2013-07-02 18:28:28', '2013-07-...",psn_app,True
3,Load_StatusPlugLoadBR2Laptop>Load_StatusSensHeatPrntADOWN,0.706849,0.712707,"{'188': [[['2014-01-05 18:29:06', '2014-01-05 21:59:06'], ['2014-01-05 18:29:06', '2014-01-05 22:29:06']]], '21': [[['2013-07-22 18:29:51', '2013-07-22 21:59:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,True
4,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildAUP,0.704110,0.704110,"{'188': [[['2014-01-05 08:29:07', '2014-01-05 18:28:06'], ['2014-01-05 10:59:07', '2014-01-05 12:59:07']]], '236': [[['2014-02-22 08:28:09', '2014-02-22 17:36:08'], ['2014-02-22 10:58:09', '2014-0...",psn_app,True
5,Load_StatusPlugLoadDesktopPCMonitor>Load_StatusSensHeatChildBUP,0.983562,0.983562,"{'188': [[['2014-01-05 08:29:07', '2014-01-05 18:28:06'], ['2014-01-05 12:29:07', '2014-01-05 12:59:07']]], '21': [[['2013-07-22 18:29:51', '2013-07-22 22:29:51'], ['2013-07-22 18:29:51', '2013-07...",psn_app,True
6,Load_StatusPlugLoadMBRTV>Load_StatusSensHeatPrntBUP,0.843836,0.843836,"{'188': [[['2014-01-05 21:59:06', '2014-01-05 23:29:06'], ['2014-01-05 21:59:06', '2014-01-05 23:58:06']]], '21': [[['2013-07-22 06:14:52', '2013-07-22 06:59:52'], ['2013-07-22 06:29:52', '2013-07...",psn_app,False
7,Load_StatusPlugLoadLRTV>Load_StatusSensHeatPrntBDOWN,0.561644,0.566298,"{'21': [[['2013-07-22 06:29:52', '2013-07-22 08:24:52'], ['2013-07-22 06:59:52', '2013-07-22 08:29:52']]], '1': [[['2013-07-02 06:28:29', '2013-07-02 08:23:29'], ['2013-07-02 06:58:29', '2013-07-0...",psn_app,False
8,Load_StatusPlugLoadLRTV>Load_StatusSensHeatPrntADOWN,0.841096,0.848066,"{'188': [[['2014-01-05 18:09:06', '2014-01-05 22:39:06'], ['2014-01-05 18:29:06', '2014-01-05 22:29:06']]], '21': [[['2013-07-22 06:29:52', '2013-07-22 08:24:52'], ['2013-07-22 06:29:52', '2013-07...",psn_app,False
9,Load_StatusPlugLoadLRTV>Load_StatusSensHeatChildADOWN,0.986301,0.994475,"{'188': [[['2014-01-05 08:29:07', '2014-01-05 10:29:07'], ['2014-01-05 08:59:07', '2014-01-05 10:59:07']], [['2014-01-05 18:09:06', '2014-01-05 22:39:06'], ['2014-01-05 19:29:06', '2014-01-05 20:1...",psn_app,False


## Second, look at rules in level3 that could be optimised and append to list

In [24]:
redundancy = pd.DataFrame()
redundancy = pd.concat([redundancy, psn_app_app_df.loc[[1425,1426,1438,1439,1477,1478,1482,1486,1506,1507,2255,2256,2268,2269,2275,2281,2311,2312,2373,2374,2384,2385]]], ignore_index=True, axis=0)
redundancy

,pattern,supp,conf,time,rule,multi_floor
0,Load_StatusBR2Lights>Load_StatusPlugLoadBR2Laptop*Load_StatusPlugLoadBR2Laptop>Load_StatusSensHeatChildADOWN*Load_StatusBR2Lights>Load_StatusSensHeatChildADOWN,0.704110,0.709945,"{'268': [[['2014-03-26 18:18:07', '2014-03-26 21:58:07'], ['2014-03-26 18:28:07', '2014-03-26 21:58:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:19:07', '2014-0...",psn_app_app,True
1,Load_StatusBR2Lights>Load_StatusPlugLoadBR2Laptop*Load_StatusPlugLoadBR2Laptop>Load_StatusSensHeatChildBDOWN*Load_StatusBR2Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'268': [[['2014-03-26 18:18:07', '2014-03-26 21:58:07'], ['2014-03-26 18:28:07', '2014-03-26 21:58:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:19:07', '2014-0...",psn_app_app,True
2,Load_StatusBR2Lights>Load_StatusPlugLoadBR3Laptop*Load_StatusPlugLoadBR3Laptop>Load_StatusSensHeatChildADOWN*Load_StatusBR2Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'268': [[['2014-03-26 18:18:07', '2014-03-26 21:58:07'], ['2014-03-26 18:28:07', '2014-03-26 20:13:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:19:07', '2014-0...",psn_app_app,True
3,Load_StatusBR2Lights>Load_StatusPlugLoadBR3Laptop*Load_StatusPlugLoadBR3Laptop>Load_StatusSensHeatChildBDOWN*Load_StatusBR2Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'268': [[['2014-03-26 18:18:07', '2014-03-26 21:58:07'], ['2014-03-26 18:28:07', '2014-03-26 20:13:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:19:07', '2014-0...",psn_app_app,True
4,Load_StatusBR3Lights>Load_StatusDRLights*Load_StatusDRLights>Load_StatusSensHeatChildADOWN*Load_StatusBR3Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 20:28:07'], ['2014-03-26 19:13:07', '2014-03-26 20:28:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:29:07', '2014-0...",psn_app_app,True
5,Load_StatusBR3Lights>Load_StatusDRLights*Load_StatusDRLights>Load_StatusSensHeatChildBDOWN*Load_StatusBR3Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 20:28:07'], ['2014-03-26 19:13:07', '2014-03-26 20:28:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:29:07', '2014-0...",psn_app_app,True
6,Load_StatusBR3Lights>Load_StatusSensHeatChildADOWN*Load_StatusSensHeatChildADOWN->Load_StatusBA2Lights*Load_StatusBR3Lights>Load_StatusBA2Lights,0.564384,0.569061,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 20:28:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07'], ['2014-03-26 20:13:07', '2014-03-26 20:23:07']]], '44': [[['2013-08-14 18:29:36', '2013-08...",psn_app_app,True
7,Load_StatusBR3Lights>Load_StatusSensHeatChildBDOWN*Load_StatusSensHeatChildBDOWN->Load_StatusBA2Lights*Load_StatusBR3Lights>Load_StatusBA2Lights,0.564384,0.569061,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 20:28:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07'], ['2014-03-26 20:13:07', '2014-03-26 20:23:07']]], '44': [[['2013-08-14 18:29:36', '2013-08...",psn_app_app,True
8,Load_StatusBR3Lights>Load_StatusPlugLoadBR3Laptop*Load_StatusPlugLoadBR3Laptop>Load_StatusSensHeatChildADOWN*Load_StatusBR3Lights>Load_StatusSensHeatChildADOWN,0.706849,0.712707,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 20:28:07'], ['2014-03-26 18:28:07', '2014-03-26 20:13:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:29:07', '2014-0...",psn_app_app,True
9,Load_StatusBR3Lights>Load_StatusPlugLoadBR3Laptop*Load_StatusPlugLoadBR3Laptop>Load_StatusSensHeatChildBDOWN*Load_StatusBR3Lights>Load_StatusSensHeatChildBDOWN,0.704110,0.709945,"{'268': [[['2014-03-26 18:28:07', '2014-03-26 20:28:07'], ['2014-03-26 18:28:07', '2014-03-26 20:13:07'], ['2014-03-26 19:28:07', '2014-03-26 20:13:07']]], '186': [[['2014-01-03 18:29:07', '2014-0...",psn_app_app,True
